# Monte Carlo Power Analysis Demo

This notebook demonstrates how to use `notebooks/analysis.py` to run Monte Carlo power analyses
for both **paired** and **unpaired** experimental designs using synthetic pilot data.

We treat the *smallest effect size of interest* (SESOI) as the AUC difference we care about,
and use pilot data to estimate variability, then simulate many experiments to estimate power
(i.e., the probability of rejecting the null when the true effect is the SESOI).


In [6]:
import sys
import pathlib
import numpy as np

# Try to import the analysis module that lives in notebooks/analysis.py
try:
    import analysis  # type: ignore[import]
except ImportError:
    repo_root = pathlib.Path.cwd().resolve()
    notebooks_dir = repo_root / "notebooks"
    if (notebooks_dir / "analysis.py").exists():
        sys.path.insert(0, str(notebooks_dir))
        import analysis  # type: ignore[import]
    else:
        raise RuntimeError(
            f"Could not locate analysis.py; tried {notebooks_dir / 'analysis.py'}"
        )

np.random.seed(42)

analysis.PowerSpec, analysis.BootstrapSpec, analysis.CompareTool


(analysis.PowerSpec, analysis.BootstrapSpec, analysis.CompareTool)

In [18]:
# Paired design: pilot differences and Monte Carlo power

# Configure SESOI and Monte Carlo parameters
power_paired = analysis.PowerSpec(
    enabled=True,
    target_effect_size=0.5,   # SESOI: mean paired difference (candidate - control)
    target_power=0.8,
    n_sim=2000,
    sample_sizes=[5, 10, 15, 20, 40, 100, 200],
    normality_alpha=0.05,
    use_bootstrap=True,
)

bootstrap_paired = analysis.BootstrapSpec(
    alpha=0.05,
    method="bca",
    side="greater",  # we care about candidate > control
)

# Instantiate the CompareTool with minimal dummy IDs so validation passes.
# We won't call .invoke(), only the internal _power_info helper.
compare_paired = analysis.CompareTool(
    control_run_ids=["pilot"],
    candidate_run_ids=["pilot"],
    power=power_paired,
    bootstrap=bootstrap_paired,
)

# Synthetic "pilot" paired differences (candidate - control)
# Here we imagine we observed ~10 seeds with mean close to SESOI and some noise.
pilot_diffs = np.random.normal(loc=0.2, scale=1.0, size=10)

power_info_paired = compare_paired._power_info("paired", pilot_diffs)

print("Paired design Monte Carlo power estimates:")
for pt in power_info_paired["curve"]:
    print(f"  N={pt['N']:2d}, estimated power={pt['power']:.3f}")

power_info_paired


Paired design Monte Carlo power estimates:
  N= 5, estimated power=0.193
  N=10, estimated power=0.329
  N=15, estimated power=0.464
  N=20, estimated power=0.551
  N=40, estimated power=0.838
  N=100, estimated power=0.995
  N=200, estimated power=1.000


{'curve': [{'N': 5, 'power': 0.193},
  {'N': 10, 'power': 0.3285},
  {'N': 15, 'power': 0.4635},
  {'N': 20, 'power': 0.5515},
  {'N': 40, 'power': 0.838},
  {'N': 100, 'power': 0.9945},
  {'N': 200, 'power': 1.0}],
 'alpha': 0.05,
 'target_power': 0.8,
 'target_effect_size': 0.5,
 'n_sim': 2000.0}

In [19]:
# Unpaired design: pilot control and candidate samples and Monte Carlo power

power_unpaired = analysis.PowerSpec(
    enabled=True,
    target_effect_size=0.2,   # SESOI: mean difference between groups
    target_power=0.8,
    n_sim=2000,
    sample_sizes=[5, 10, 15, 20, 40, 100, 200],
    normality_alpha=0.05,
    use_bootstrap=True,
)

bootstrap_unpaired = analysis.BootstrapSpec(
    alpha=0.05,
    method="bca",
    side="greater",
)

# Instantiate the CompareTool with minimal dummy IDs so validation passes.
# Again, we only call _power_info, not .invoke().
compare_unpaired = analysis.CompareTool(
    control_run_ids=["pilot_control"],
    candidate_run_ids=["pilot_candidate"],
    power=power_unpaired,
    bootstrap=bootstrap_unpaired,
)

mu_control = 0.0
sesoi = 0.2
sd = 1.0

# Synthetic pilot data for each arm
pilot_control = np.random.normal(loc=mu_control,         scale=sd, size=20)
pilot_candidate = np.random.normal(loc=mu_control+sesoi, scale=sd, size=20)

power_info_unpaired = compare_unpaired._power_info("unpaired", pilot_control, pilot_candidate)

print("Unpaired design Monte Carlo power estimates:")
for pt in power_info_unpaired["curve"]:
    print(f"  N={pt['N']:2d}, estimated power={pt['power']:.3f}")

power_info_unpaired


Unpaired design Monte Carlo power estimates:
  N= 5, estimated power=0.089
  N=10, estimated power=0.121
  N=15, estimated power=0.140
  N=20, estimated power=0.154
  N=40, estimated power=0.212
  N=100, estimated power=0.398
  N=200, estimated power=0.656


{'curve': [{'N': 5, 'power': 0.0895},
  {'N': 10, 'power': 0.1215},
  {'N': 15, 'power': 0.1395},
  {'N': 20, 'power': 0.154},
  {'N': 40, 'power': 0.2125},
  {'N': 100, 'power': 0.398},
  {'N': 200, 'power': 0.656}],
 'alpha': 0.05,
 'target_power': 0.8,
 'target_effect_size': 0.2,
 'n_sim': 2000.0}